In [1]:
import requests
from bs4 import BeautifulSoup as bs
from collections import defaultdict
import pandas as pd
from xml.etree import ElementTree
import requests
import random

In [2]:
# # Code to open the pkl file:
# first30Kdf = pd.read_pickle("pubMedFirst30kEntriesw4CitationStyles.pkl")
# first30Kdf.sample(10)

In [3]:
# # ground truth part

# rename pubmed22n0001 with whichever chunk you want to parse

# dom = ElementTree.parse("pubmed22n0001.xml")
# Articles = dom.findall('PubmedArticle/MedlineCitation/Article')
# pubmedID = dom.findall('PubmedArticle/PubmedData/ArticleIdList/')
# allTitles = []
# allAuthors = []
# # indexes = []
# IDs = []

# for i, eachArticles in enumerate(Articles):
#     # index.append(i+1)
#     title = eachArticles.find('ArticleTitle').text
#     allTitles.append(title)
#     allArticleAuthors = []
#     # for eachId in eachArticles.findall()
#     for authors in eachArticles.findall('AuthorList/Author'):
#         if authors.find('LastName') is not None:
#             LastName = authors.find('LastName').text
#         else:
#             LastName = "NULL"
#         if authors.find('ForeName') is not None:
#             ForeName = authors.find('ForeName').text
#         else:
#             ForeName = "NULL"
#         allArticleAuthors.append(ForeName + " " + LastName)
#     allAuthors.append(allArticleAuthors)
# for i, each in enumerate(pubmedID):
#     # indexes.append(i+1)
#     if (each.attrib == {'IdType': 'pubmed'}) is not None:
#         if each.attrib == {'IdType': 'pubmed'}:
#             IDs.append(each.text)
#             # indexes.append(i+1)
#     else:
#         IDs.append("NULLID")
#         # indexes.append(i+1)

# components = {
#     'Title' : allTitles,
#     'Author': allAuthors,
#     # 'Indexes' : indexes,
#     'PubMedIDs' : IDs
# }

# bigDf = pd.DataFrame(components)

In [4]:
# # perfect formatted with parallel approach, it wilk take 3-4 mins for 30k, with the following approach, about 30 mins
# customString = "https://pubmed.ncbi.nlm.nih.gov/"
# endCString = "/citations/"
# styles = ["ama", "apa", "mla", "nlm"]

# ama, apa, mla, nlm = [], [], [], []
# # for i, eachId in enumerate(IDs):
# errorList = []
# for eachId in IDs:
#     try:
#         tempMatch = requests.get(customString + eachId + endCString).text
#         matchDict = eval(tempMatch)
#         try:
#             ama.append(matchDict["ama"]['orig'])
#         except:
#             ama.append("NULL")
#         try:
#             apa.append(matchDict["apa"]['orig'])
#         except:
#             apa.append("NULL")
#         try:
#             mla.append(matchDict["mla"]['orig'])
#         except:
#             mla.append("NULL")
#         try:
#             nlm.append(matchDict["nlm"]['orig'])
#         except:
#             nlm.append("NULL")
#     except:
#         errorList.append(eachId)

# bigDf = bigDf[~bigDf['PubMedIDs'].isin(errorList)]
# bigDf.reset_index(inplace=True)
# rawStringDf = pd.DataFrame(list(zip(ama,apa,mla,nlm)), columns = ["ama","apa","mla","nlm"])
# df = pd.concat([bigDf, rawStringDf],axis=1)
# df.drop(["index"], axis = 1, inplace=True)
# df.to_pickle("first30k.pkl")

In [5]:
# Human made part:
customString = "https://pubmed.ncbi.nlm.nih.gov/"
refString = "/references/"
holderIDs = random.sample(range(1, 33000000), 1000)
# holderIDs = [x for x in range(1,30000)] # make changes here to adjust to desired chunk
allHumanMadeReference = defaultdict(list)
errorList = []
IDsWOCitation = [] # for statistics
for eachID in holderIDs:
    try:
        tempRawCitationPage = requests.get(customString + str(eachID) + refString).text
        soup = bs(tempRawCitationPage, "html.parser")
        tempCitedByList = soup.find_all('li', class_= 'skip-numbering')
        if len(tempCitedByList) == 0:
            IDsWOCitation.append(eachID)
            continue # skip the operation
        for each in tempCitedByList: # tempCitedByList has all references on that article
            # there are cases where there are more than 1 anchor tag per entry
            tempHrefCheck = each.find_all('a', href = True)
            if(tempHrefCheck is not None):
                for eachTempHrefCheck in tempHrefCheck:
                    # href could be in the form of DOI url or hyperlink, filter just hyperlink, strip then use as key(pubmedID)
                    if len(eachTempHrefCheck["href"]) <= 10: # total around 33M, 33M has 8 char plus 2x "/"
                        allHumanMadeReference[int(eachTempHrefCheck["href"][1:-1])].append(each.contents[0].strip().split("\n")[0])
    except:
        errorList.append(eachID)

In [6]:
errorList

[]

In [7]:
len(IDsWOCitation)

924

In [8]:
allHumanMadeReference

defaultdict(list,
            {30207593: ['Bray F, Ferlay J, Soerjomataram I, et al. Global cancer statistics 2018: GLOBOCAN estimates of incidence and mortality worldwide for 36 cancers in 185 countries. CA Cancer J Clin. 2018;68:394–424. doi:10.3322/caac.21492',
              'Freddie B, Jacques F, Isabelle S, et al. Global cancer statistics 2018: GLOBOCAN estimates of incidence and mortality worldwide for 36 cancers in 185 countries. CA Cancer J Clin. 2018;68(6):394–424. doi:10.3322/caac.21492'],
             26742998: ['Siegel RL, Miller KD, Jemal A. Cancer statistics, 2016. CA Cancer J Clin. 2016;66:7–30. doi:10.3322/caac.21332'],
             24120480: ['Nordlinger B, Sorbye H, Glimelius B, et al. Perioperative FOLFOX4 chemotherapy and surgery versus surgery alone for resectable liver metastases from colorectal cancer (EORTC 40983): long-term results of a randomised, controlled, Phase 3 trial. Lancet Oncol. 2013;14:1208–1215. doi:10.1016/S1470-2045(13)70447-9'],
             1272